# PB016: Artificial intelligence I, labs 11 - Machine learning

Today we'll dig into machine learning, or rather two specific examples of "classic" machine learning models. We'll focus namely on:
1. __Representation and preparation of input data__
2. __Simple algorithm for training a perceptron__
3. __Simple algorithm for training a decision tree__
4. __The exercises in a few lines of code__

An __important note__: from these labs on, all your work on the assignments will be collaborative - similarly to the last part of labs 10, split into groups (min 2, max 4 people) and work together.

---

## 1. Representation and preparation of input data

__Basic facts__
- Machine learning typically works with a training set of machine-readable input data in the form of examples, which may (or may not) be labeled by an identifier (or identifiers) of the class (or classes) to which they belong.
- Each example is usually assigned a unique identifier and a set of specific property (or also feature) values that depend on the specific domain of the problem.
- A suitable representation of such data is a matrix mapping the example identifiers to vectors of their specific feature values, and (for supervised learning) also a vector mapping the example identifiers to the identifiers of the classes to which they belong.

### Sample problem - classification of [black metal](https://en.wikipedia.org/wiki/Black_metal) fans

![black_metal](https://www.fi.muni.cz/~novacek/courses/pb016/labs/img/blackmetal.png)

- The problem may not be as simple as it seems (for example, not all black metal fans are Satanists with striking body paint).
- Example of a simple dataset:

| __ID__ || __HL__ | __FHT__ | __LN__ | __LS__ || __FBM__ |
|--------||--------|---------|--------|--------||---------|
| Petr   || l      | br      | 9/10   | 1/10   || y       |
| Marie  || s      | $-$     | 8/10   | 9/10   || y       |
| Azazel || l      | ms      | 1/10   | 10/10  || n       |
| Pavel  || b      | br      | 7/10   | 2/10   || y       |

- Meaning of columns and individual feature values (with the "$ - $" symbol for "missing"):
  - __ID__ - example identifier (names of people we classify into "black metal fan" and "others" classes)
  - __HL__ - hair length (one of $ \{b, s, m, l \} $ for bald, short, medium and long hair)
  - __FHT__ - beard type (one of $ \{no, gt, ms, sb, br \} $ for no beard, goatie, moustache, sideburns and beard)
  - __LN__ - relationship to nature (on a scale from 1 to 10, the higher the more devout)
  - __LS__ - relationship to Satan (on a scale from 1 to 10, the higher the more devout)
  - __FBM__ - belonging to the class of black metal fans (one of $ \{y, n \} $ for yes and no)

### Simple structures for sample data representation

In [ ]:
# vectors (in the form of dictionaries) mapping the symbolic identifiers of the
# examples and features to unique numbers

example_id_map = {
  'Petr' : 0,
  'Marie' : 1,
  'Azazel' : 2,
  'Pavel' : 3,
}

feature_id_map = {
  'HL' : 0,
  'FHT' : 1,
  'LN' : 2,
  'LS' : 3,
}

# inverse vectors to the previous two (may come handy for user-friendly
# statements)

example_id_map_inv = dict([(y,x) for x,y in example_id_map.items()])
feature_id_map_inv = dict([(y,x) for x,y in feature_id_map.items()])

# vector mapping example identifiers to identifiers of their classes
# (0 corresponds to 'n', 1 corresponds to 'y')

labels = {0:1, 1:1, 2:0, 3:1}

# matrix (in the form of a "two-dimensional" dictionary) mapping example
# identifiers to the values of their properties (values are currently
# initialized to 0)

features = {}
for i in range(4):
  for j in range(4):
    features[(i,j)] = 0

### __Numerical representation of property values__

- Many machine learning algorithms work better with "nice and clean" numerical data.
- Let's assign such values to the individual fields of the `features` matrix based on the information in the problem definition above.
- Word of warning - it's not as trivial as it might seem:
 - Numeric values should ideally preserve the meaning of the original ones.
   - In our example, this concerns primarily the facial hair type - while zero may represent no facial hair and higher numerical values may correspond to increasingly "bushy" facial hair (as it's done below), in practice, one would typically consider such features [categorical](https://medium.com/@nikhiloswal/categorical-features-in-machine-learning-14ed910c0812) and choose for instance [one-hot encoding](https://en.wikipedia.org/wiki/One-hot#Machine_learning_and_statistics) for their representation in the feature matrix.
 - Missing data items should typically be meaningfully filled in ("imputed").
 - For many machine learning algorithms, the data should also be normalised so that the ranges of the individual columns are more or less balanced.

#### __Possible numerical representation of property values__

In [ ]:
# feature value ranges and their corresponding numerical representations:
# HL:  {b,s,m,l}        ~ {0,1,2,3}
# FHT: {no,gt,ms,sb,br} ~ {0,1,2,3,4}
# LN:  1..10            ~ 1..10
# LS:  1..10            ~ 1..10

# numerical representation of property values - training data set
# (note: imputation uses domain knowledge - women usually do not have much
#  facial hair)

features = {
  (0,0) : 3, (0,1) : 4, (0,2) : 9, (0,3) : 1,  # ~ Petr:   l, br, 9/10, 1/10
  (1,0) : 1, (1,1) : 0, (1,2) : 8, (1,3) : 9,  # ~ Marie:  s, -, 9/10, 9/10
  (2,0) : 3, (2,1) : 2, (2,2) : 1, (2,3) : 10, # ~ Azazel: l, ms, 1/10, 10/10
  (3,0) : 0, (3,1) : 4, (3,2) : 7, (3,3) : 2,  # ~ Pavel:  b, br, 7/10, 2/10
}

# normalization of property values (by maximum of each column)

# initial maximum value
max_dict = {0:0, 1:0, 2:0, 3:0}

# determining the actual value of the maximum by scanning the matrix of
# property values
#   - NOTE: this implementation finds the maximum from the data, which may not
#     be completely correct for the relationship to nature, where the "real"
#     maximum (10) is not in the data (the maximum there is 9)
for i,j in features:
  if features[(i,j)] > max_dict[j]:
    max_dict[j] = features[(i,j)]

# normalization of values to 0-1 interval by dividing the values by the maximum
for i,j in features:
  features[(i,j)] /= max_dict[j]

# listing the final matrix
rows = []
for i in range(4):
  columns = []
  for j in range(4):
    columns.append(round(features[(i,j)],1))
  rows.append('   '.join([str(x) for x in columns]))
print('Normalized matrix of numeric property values: \n')
print('\n'.join(rows))

---

## 2. A simple algorithm for training a perceptron

__Basic facts__
- [Perceptron](https://en.wikipedia.org/wiki/Perceptron) is one of the oldest concepts in the theory (and practice) of machine learning (proposed in 1958).
- It is a [linear](https://en.wikipedia.org/wiki/Linear_classifier) ​​[binary](https://en.wikipedia.org/wiki/Binary_classification) classifier for supervised learning over data represented by number vectors.
- It is based on the so-called threshold function $ f $ mapping the input vector $ \mathbf{x} $ to a binary scalar $ f(\mathbf{x}) $ as follows:
 - $ f(\mathbf{x}) = 1 $ if $ \mathbf{w} \cdot \mathbf{x} + b \geq 0 $
 - otherwise $ f(\mathbf{x}) = 0 $
- This function has two parameters - a scalar threshold $ b $ (can be equal to zero) and a weight vector $ \mathbf{w} $, which is optimized to best discriminate the data based on their membership in individual classes (positive vs. negative examples).
- The perceptron has become the basis of modern [deep learning](https://en.wikipedia.org/wiki/Deep_learning), which (very simply) uses perceptron-like units in many complex interconnected "hidden" layers. Parameters (i.e., weight vectors) are usually optimized by [backpropagation](https://en.wikipedia.org/wiki/Backpropagation) in such networks.

__General perceptron for illustration__

![perceptron](https://www.fi.muni.cz/~novacek/courses/pb016/labs/img/perceptron.png)

### Rules for incremental updating of scales when learning the perceptron
- For simplicity, assume that the parameter $ b = 0 $.
- At the beginning we initialize the vector $ \mathbf{w} $ to random values.
- In each individual learning step ("epoch") we randomly select one example $ \mathbf{x} $ from the training dataset.
 - If the example $ \mathbf{x} $ is positive and $ \mathbf{w} \cdot \mathbf{x} < 0 $, we assign $ \mathbf{w} $ the value $ \mathbf{w} + \mathbf{x} $.
 - If the example $ \mathbf{x} $ is negative and $ \mathbf{w} \cdot \mathbf{x}> 0 $, we assign $ \mathbf{w} $ the value $ \mathbf{w} - \mathbf{x} $.
- We repeat this until an approximate convergence is achieved (e.g., until all examples are classified correctly or until a predefined number of epochs is reached).

### Auxiliary functions for vector operations to be used in the following assignments

In [ ]:
# auxiliary vector operation functions

def dot_product(u,v):
  """Computing the dot product of the two input vectors.

  Parameters
  ----------
  u, v : dict
      The vectors for which the dot product is computed (represented as
      dictionaries mapping indices of the vector elements to the element
      values).

  Returns
  -------
  float
      The value of the dot product of the two input vectors.
  """

  if len(u) != len(v):
    raise ValueError
  dp = 0
  for i in range(len(u)):
    dp += u[i]*v[i]
  return dp

def plus(u,v):
  """Adding up the two input vectors.

  Parameters
  ----------
  u, v : dict
      The vectors which are to be added up (represented as dictionaries mapping
      indices of the vector elements to the element values).

  Returns
  -------
  dict
      The vector resulting from the addition of the input vectors.
  """

  if len(u) != len(v):
    raise ValueError
  w = u
  for i in range(len(w)):
    w[i] += v[i]
  return w

def minus(u,v):
  """Subtracting the second input vector from the first.

  Parameters
  ----------
  u, v : dict
      The vectors which are to be subtracted (represented as dictionaries
      mapping indices of the vector elements to the element values).

  Returns
  -------
  dict
      The vector resulting from the subtraction of the second input vector from
      the first.
  """

  if len(u) != len(v):
    raise ValueError
  w = u
  for i in range(len(w)):
    w[i] -= v[i]
  return w

### __Exercise 2.1: Training a simple perceptron__

- Implement the above algorithm, training on and classifying the black metal data set we worked with in the previous section.
- Experiment with convergence as you wish.
- A note on the solution - use the predefined auxiliary functions for operations with vectors represented by dictionaries in Python (in the spirit of the previous section).

In [ ]:
import random

# learning the weight vector (main part of the algorithm; "convergence"
# according to a fixed number of epochs)

def optimise_weights(n_epochs=20):
  """Optimising a pre-defined weight vector of length four corresponding to the
  method demonstration (initialised to random values).

  Parameters
  ----------
  n_epochs : int
      The number of epochs for which the weight vector is to be optimised.

  Returns
  -------
  dict
      The optimised weight vector.
  """

  # random weight init - TODO - COMPLETE YOURSELF

  w = dict([(i,0) for i in range(4)])

  # iterative updates of the weights

  for epoch in range(n_epochs):

    # TODO - COMPLETE YOURSELF

    pass # TODO - REMOVE IN THE ACTUAL IMPLEMENTATION

  return w

# function for verifying the classification of training examples based on the
# weight vector

def classify(x_id,w):
  """Classifying an example based on the weight vector.

  Parameters
  ----------
  x_id : int
      The ID of the example to be classified.
  w : dict
      The weight vector.

  Returns
  -------
  int
      0 for negative examples and 1 for positive examples.
  """

  # creating the example vector
  x = {}
  for j in range(4):
    x[j] = features[(x_id,j)]

  # multiplying the vector by the weights
  if dot_product(x,w) > 0:
    return 1
  return 0

In [ ]:
# testing the implementation
w = optimise_weights()
print('Learned weight vector:', ' '.join([str(w[i]) for i in range(4)]))
print('Classifying the training examples...')
for i in range(4):
  class_i = classify(i,w)
  print('Example:', example_id_map_inv[i])
  print('  actual class   :', labels[i])
  print('  predicted class:', class_i)

### __Food for thought:__

- Is the chosen data representation suitable for perceptron learning?
- What could possibly be done differently?

---

## 3. A simple algorithm for training a decision tree

__Basic facts__
- [Decision tree learning](https://en.wikipedia.org/wiki/Decision_tree_learning) is one of the most widely used types of algorithms in data analysis.
- It is based on the recursive division of the input set of examples into as homogeneous subsets as possible (in terms of belonging to individual classes according to the labeling of training examples).
- The division of the "parent" takes place on a parameter that maximizes the homogeneity of the "offspring."
- This recursive division according to the values ​​of individual features corresponds to the decision (i.e., non-leaf) nodes of the tree.
- The leaves, on the other hand, typically contain data that most likely belong to one particular class (note: in this exercise, for simplicity, we only deal with classification trees, not regression trees that associate the individual examples with numerical values ​​on a continuous scale).
- The resulting structure is then used to classify data for which we do not know the individual classes.

__A sample decision tree:__

![decision_tree](https://www.fi.muni.cz/~novacek/courses/pb016/labs/img/decision_tree.jpg)

### Rules for dividing the datasets when creating a decision tree
- There are a number of metrics for this (see for instance [here](https://en.wikipedia.org/wiki/Decision_tree_learning#Metrics)).
- We will deal with a specific one, the so-called [information gain](https://en.wikipedia.org/wiki/Information_gain_in_decision_trees):
 - Based on [entropy](https://en.wikipedia.org/wiki/Entropy_%28information_theory%29).
 - To divide the "parent set", we use the feature that maximizes the difference between the entropy of the parent and the weighted average entropy of the individual "offspring" (the weights correspond to the relative number of examples in each offspring).

#### Practical example - division of the root node (i.e., the whole data set) of fans and non-fans of black metal

Original dataset:

| __ID__ || __HL__ | __FHT__ | __LN__ | __LS__ || __FBM__ |
|--------||--------|---------|--------|--------||---------|
| Petr   || l      | br      | 9/10   | 1/10   || y       |
| Marie  || s      | $-$     | 8/10   | 9/10   || y       |
| Azazel || l      | ms      | 1/10   | 10/10  || n       |
| Pavel  || b      | br      | 7/10   | 2/10   || y       |

Data set after transformation:

| __ID__ || 0   | 1   | 2   | 3   || y |
|--------||-----|-----|-----|-----||---|
| 0      || 1   | 1   | 1   | 0.1 || 1 |
| 1      || 0.3 | 0   | 0.9 | 0.9 || 1 |
| 2      || 1   | 0.5 | 0.1 | 1   || 0 |
| 3      || 0   | 1   | 0.8 | 0.2 || 1 |

Parent set entropy ($P$):
- $I_E(P) = I_E([3,1]) = -\frac{3}{4}\log_2\frac{3}{4} - \frac{1}{4}\log_2\frac{1}{4} \doteq 0.811$

Information gain of the feature 0 (hair length):
- The values of the features are normalized, so we can simplify the problem to a binary division to "halves," i.e., divide the examples into subsets according to the values of the attribute $ 0 $ less than $ 0.5 $, or greater than or equal to $ 0.5 $.
- Entropy of offspring:
 - $I_E(P_{v(0) < 0.5}) = I_E([2,0]) = 0$ (both examples with a normalized hair length value less than 0.5, i.e., Marie and Pavel, are positive, so there is no degree of uncertainty in the set and the entropy is thus zero by definition)
 - $I_E(P_{v(0) \geq 0.5}) = I_E([1,1]) = -\frac{1}{2}\log_2\frac{1}{2} - \frac{1}{2}\log_2\frac{1}{2} = 1$ (Petr is a positive example, Azazel is negative, it's "one on one", so the entropy is maximal)
- Weighted mean entropy of offspring:
 - $wmean(I_E(P_{v(0) < 0.5}), I_E(P_{v(0) \geq 0.5})) = 0\cdot\frac{2}{4} + 1\cdot\frac{2}{4} = 0.5$
- Information gain:
 - $I_E(P) - wmean(I_E(P_{v(0) < 0.5}), I_E(P_{v(0) \geq 0.5})) \doteq 0.311$

Similarly for features $ 1,2,3 $ (note: does anyone already know who "wins"?) ...

### Entropy calculation for training a decision tree

- Let's implement a function for calculating the entropy of a set of examples based on their individual class labels.
- For demonstration purposes, we will only use the data structures defined above for the "black metal" dataset.

In [ ]:
from math import log

# function returning the entropy of the proportions of positive and negative
# examples in a given data subset

def entropy(example_ids):
  """Computing the entropy of the proportions of positive and negative
  examples in a given data subset (represented by the example identifiers,
  i.e., the indices of the rows in the feature matrix and label vectors defined
  earlier in this notebook).

  Parameters
  ----------
  example_ids : list
      The identifiers of the examples for which the entropy is to be computed.

  Returns
  -------
  float
      The entropy of the proportions of positive and negative examples in the
      given data subset.
  """

  # determining the number of positive and negative examples in a set

  n_pos, n_neg = 0, 0
  for example_id in example_ids:
    if labels[example_id] == 1:
      n_pos += 1
    else:
      n_neg += 1

  # total number of examples

  n_tot = n_pos+n_neg

  # computing the entropy

  if n_pos == 0 or n_neg == 0:
    return 0
  return -(n_pos/n_tot)*log(n_pos/n_tot,2) - (n_neg/n_tot)*log(n_neg/n_tot,2)

In [ ]:
# random implementation test - Petr (0) and Marie (1) belong to the same class,
# but Marie and Azazel (2) do not
assert entropy([0,1]) == 0
assert entropy([1,2]) == 1

### __Exercise 3.1: Computing all subsets for determining the optimal dataset distribution__

- Use the entropy calculation function to implement a function that computes the information gain of possible distributions of the input data set.
- When implementing, use the procedure from the example above (including the simplifications, such as splitting the set to "halves" according to the values of the given property).

An __important note__ - bear in mind what are the sets that are being split. It might help to recall how the data looks like, and what does it correspond to in the code.

This is how the `features` data structure looks like:

| __ID__ || 0   | 1   | 2   | 3   |
|--------||-----|-----|-----|-----|
| 0      || 1   | 1   | 1   | 0.1 |
| 1      || 0.3 | 0   | 0.9 | 0.9 |
| 2      || 1   | 0.5 | 0.1 | 1   |
| 3      || 0   | 1   | 0.8 | 0.2 |

The row IDs correspond to Petr, Marie, Azazel, Pavel, respectively. The column IDs correspond to the hair length, facial hair type, love of nature, love of Satan features, respectively. The parent set from the information gain example above corresponds to a list `[0,1,2,3]` (all row IDs, i.e., all training examples). The feature we are splitting the data set on is `0` (column ID corresponding to the hair length). The resulting splits are `[[1,3], [0,2]]` (the example IDs for which the value of the feature `0` is below or above 0.5, respectively) - this is what `splits[0]['splits']` would be set to in the code below. The corresponding `splits[0]['ig']` value would be ca. 0.311.

Finally, this is how the `labels` data structure looks like:

| __ID__ || y |
|--------||---|
| 0      || 1 |
| 1      || 1 |
| 2      || 0 |
| 3      || 1 |

Once again, the row IDs correspond to Petr, Marie, Azazel, Pavel, respectively. This is what is being used to compute the entropy of a set of training example IDs - the pre-defined `entropy` function looks up the labels of each ID in the input list and computes the probabilities of picking positive and negative examples from the text, from which the overall set entropy is computed.

In [ ]:
# computing subsets for optimal distribution of a set according to the
# information gain of individual properties

def get_splits(example_ids):
  """Computing the splits of the input set of examples (represented by the
  example identifiers, i.e., the indices of the rows in the feature matrix and
  label vectors defined earlier in this notebook). A split per each of the
  four possible features is performed.

  Parameters
  ----------
  example_ids : list
      The identifiers of the examples for which the splits are to be computed.

  Returns
  -------
  dict
      The splits of the input set of examples, along with their information
      gain. The structure of the output dictionary is as follows:
      - the keys are the fearure identifies (i.e., column identifiers in the
        feature matrix defined earlier in this notebook)
      - the values are nested dictionaries that store the following:
        - a list of two lists (indexed by `'splits'`) - two non-overlapping
          subsets of the input IDs resulting from the input split based on the
          value of the corresponding feature being lower than 0.5 or not,
          respectively
        - a float number (indexed by `'ig'`) reflecting the information gain of
          this particular split
  """

  # distributions and their information gain - initialization

  splits = {
    0 : {'splits' : [], 'ig' : 0}, # splits by feature 0 (hair length)
    1 : {'splits' : [], 'ig' : 0}, # splits by feature 1 (facial hair)
    2 : {'splits' : [], 'ig' : 0}, # splits by feature 2 (love of nature)
    3 : {'splits' : [], 'ig' : 0}  # splits by feature 2 (love of Satan)
  }

  # TODO - COMPLETE THE REST YOURSELF

  return splits

In [ ]:
# listing the distribution of the entire dataset (tree root node); sorted in a
# descending order by information gain
splits = get_splits([0,1,2,3])
for feature_id in sorted(splits,key=lambda x: splits[x]['ig'],reverse=True):
  print('Splitting by feature %d with information gain=%f:' % \
        (feature_id,splits[feature_id]['ig']))
  print('  ', splits[feature_id]['splits'])

### __Exercise 3.2: Learning the actual decision tree__

- Implement a decision tree learning function that:
 - Recursively calls the function for computing the information gain of possible distributions of the input data set.
 - Selects the optimal distribution according to the most discriminating features.
 - Stores information about the parents of individual nodes as a representation of the resulting tree.
- Notes on the solution:
 - Feel free to implement the tree simply as a dictionary mapping nodes to their parents.
 - The non-leaf nodes can be strings representing what feature is the split being made on at that level.
 - Similarly, the leaves can simply be strings saying whether they're positive or negative example nodes, and listing the corresponding example IDs.
 - For instance, a tree with a root `split_on_feature_X_at_level_0` and two immediate leaves `pos : [0,1], neg : [2,3]` as its children can be represented as a Python dictionary `{'split_on_feature_X_at_level_0' : None, 'pos : [0,1]' : 'split_on_feature_X_at_level_0', 'neg : [2,3]' : 'split_on_feature_X_at_level_0'}`

In [ ]:
# checking the homogeneity of a data set (important for stopping the recursion)

def is_pure(example_ids):
  """Checking the homogeneity of the input example set (represented by the
  example identifiers, i.e., the indices of the rows in the feature matrix and
  label vectors defined earlier in this notebook).

  Parameters
  ----------
  example_ids : list
      The identifiers of the examples for which the homogeneity is to be
      checked.

  Returns
  -------
  bool
      `True` for a set that contains examples labeled with the same class,
      `False` for a set that contains examples labeled with different classes.
  """

  # TODO - COMPLETE YOURSELF

  return False # TODO - POSSIBLE CHANGE IN THE ACTUAL IMPLEMENTATION

# recursive splitting of the data set

def split_set(example_ids,parent_feature,parent_map,level):
  """Recursively splitting the set of examples IDs (represented by the example
  identifiers, i.e., the indices of the rows in the feature matrix and label
  vectors defined earlier in this notebook). Updates the mapping of node
  descriptions to their parent descriptions (represented as a dictionary).

  Parameters
  ----------
  example_ids : list
      The identifiers of the examples for which the split is to be performed.
  parent_feature : str or NoneType
      The description of the parent node. `None` for the root node. String
      reflecting the name of the feature on which the split was performed at
      the preceding level of the growing tree and the depth of that level (just
      in case the same feature is being used for splits at different tree
      levels).
  parent_map : dict
      The current mapping of node descriptions to their parent descriptions.
  level : int
      The depth of the current level of the growing tree.

  Returns
  -------
  dict
      The mapping of node descriptions to their parent descriptions.
  """

  # TODO - COMPLETE YOURSELF (RETURNS A DICTIONARY MAPPING NODES TO THEIR
  #        PARENTS)

  return parent_map

In [ ]:
# learning the tree from the complete input data
decision_tree = split_set([0,1,2,3],None,{},0)
# printing the learned tree
print('Learned decision tree:')
print(decision_tree)

### __Food for thought:__

- Is the chosen data representation suitable for learning decision trees?
- What might be done differently?
- How does the resulting representation of the classifier conceptually differ from the perceptron? For example, is there something it can do much better?

---
## 4. The exercises in a few lines of code

- There are a number of libraries and really powerful machine learning tools available in Python.
- One example is [scikit-learn](https://scikit-learn.org/) - a general library implementing de facto all "classical" machine learning algorithms.
- This library can be used for solving the above exercises in a much more systematic manner, and using much less code, as shown in the following example.



### __*Exercise 4.1: Using scikit-learn to solve the labs assignments (optional)__

- Training a perceptron model and a decision tree model on the data in this notebook using scikit-learn.
- Use the trained classifiers to classify unknown examples designed by you.

In [ ]:
# TODO - complete yourselves

---

#### _Final note_ - the materials used in this notebook are original works credited and licensed as follows:
- Sources for the picture illustrating the two faces of black metal:
 - Retrieved from Wikimedia Commons [here](https://commons.wikimedia.org/wiki/File:Gorgoroth_I.jpg) and [here](https://en.wikipedia.org/wiki/File:Celestial_Lineage_2011.jpg)
 - Authors: [Alina Sofia](https://www.flickr.com/people/43529122@N08) and [Alison Scarpulla](https://commons.wikimedia.org/w/index.php?title=User:Kingofcupsqueenofswords&action=edit&redlink=1)
 - License: [Creative Commons Attribution-Share Alike 2.0 (CC BY-SA 2.0)](https://creativecommons.org/licenses/by-sa/2.0/deed.en) and [Creative Commons Attribution-Share Alike 4.0 International ](https://creativecommons.org/licenses/by-sa/4.0/deed.en)
- Picture of the general perceptron:
 - Retrieved from [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Perceptron.svg)
 - Author: [Mat the w](https://en.wikipedia.org/wiki/User:Mat_the_w)
 - License: [Creative Commons Attribution-Share Alike 3.0 Unported](https://creativecommons.org/licenses/by-sa/3.0/deed.en)
- Picture of the decision tree:
 - Retrieved from [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Decision_Tree.jpg)
 - Author: [Gilgold](https://commons.wikimedia.org/w/index.php?title=User:Gilgoldm&action=edit&redlink=1)
 - License: [Creative Commons Attribution-Share Alike 4.0 International](https://creativecommons.org/licenses/by-sa/4.0/deed.en)